Task 1: Label each employee message as Positive, Negative, or Neutral using VADER
1. Import libraries and Load Dataset

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

#LOAD test .csv file 
path="C:/Users/ladom/Desktop/TS/ML/AI-project-submission/"
df=pd.read_csv(f"{path}data/test(in).csv")

df.head()

2.Initialize VADER and Create  Sentiment Labeling Function

In [ ]:
# create a Vader sentiment analyzer 
analyzer=SentimentIntensityAnalyzer()

# Define a function to get sentiment score
def get_sentiment_label(text):
    score=analyzer.polarity_scores(text)['compound']
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

3. Apply labeling and Save labeled data 

In [ ]:
df['sentiment']=df['body'].apply(get_sentiment_label)
# Save labeled dataset for further tasks 
df.to_csv(f"{path}data/labeled_test.csv")

Task 2: Eploratory Data Anlysis (EDA) 
1. Analyze labeled dataset to understand the structure 

In [ ]:
# Load labeled dataset 
df=pd.read_csv(f"{path}data/labeled_test.csv")

print(df.info())

df.head()



In [ ]:
# Check  for missing values 
df.isnull().sum()

# basic counts '
print(f"Total number of messages: {df.shape[0]}")
print(f"Total number of unique employees: {df['from'].nunique()}")



2. Sentiment Distribution 

In [ ]:

setiment_counts=df['sentiment'].value_counts()
print(setiment_counts)

plt.figure(figsize=(6,4))
sns.countplot(x='sentiment', data=df, palette={"Positive":"green", "Negative":"red", "Neutral":"blue"})
plt.title("Sentiment Label Distribution")
plt.xlabel("Sentiment")
plt.ylabel("Number of Messages")
plt.show()


3. Convert Data column to datetime , Create month period column

In [ ]:
df['date']=pd.to_datetime(df['date'])

# check the date range 
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
df['month']=df['date'].dt.to_period('M') 

# Group by month and sentiment 
monthly_sentiment=df.groupby(['month','sentiment']).size().unstack().fillna(0)
monthly_sentiment

4. Plot monthly sentiment trends (stacked bar chart )

In [ ]:
monthly_sentiment.plot(kind='bar', stacked=True, figsize=(12,6), color={
    "Positive":"green",
    "Negative":"red",
    "Neutral":"blue"
})
plt.title("Monthly Sentiment Trends")
plt.xlabel("Month")
plt.ylabel("Number of Messages")
plt.xticks(rotation=45)
plt.legend(title="Sentiment")
plt.show()


5. Plot Negative message trend Seperately 

In [ ]:
monthly_sentiment['Negative'].plot(kind='line', marker='o', color='red', figsize=(10,5))
plt.title('Trend of Negative Messages Over Time')
plt.xlabel('Month')
plt.ylabel('Number of Negative Messages')
plt.grid(True)
plt.show()


6. Save EDA summaries 

In [ ]:
monthly_sentiment.to_csv(f"{path}reports/employee_sentiments/monthly_sentiment_summary.csv")
setiment_counts.to_csv(f"{path}reports/employee_sentiments/sentiment_distribution.csv")


Task 3 .  Employee Score Calculation

In [ ]:
# Define scoring function
def sentiment_score(label):
    if label == 'Positive':
        return 1
    elif label == 'Negative':
        return -1
    else:  # Neutral
        return 0

In [ ]:
# APPLY Score to each message 
df['score'] = df['sentiment'].apply(sentiment_score) 
# Preview data with new score column 
df[['from', 'date', 'sentiment', 'score']].head()

# Group by employee and month, aggregate scores

In [ ]:
employee_monthly_score = df.groupby(['from', 'month'])['score'].sum().reset_index()
# Preview the result 
employee_monthly_score.head()

Pivot for easier visualization (employees as rows, months as column)

In [ ]:
employee_monthly_pivot = employee_monthly_score.pivot(index='from', columns='month', values='score').fillna(0)
#Preview pivot table 
employee_monthly_pivot.head()

# Visualize sample employee score trends (for top 5 employees alphabetically)

In [ ]:
sample_employees = employee_monthly_pivot.index[:5]
plt.figure(figsize=(12,6))
for emp in sample_employees:
    sns.lineplot(
        x=employee_monthly_pivot.columns.astype(str),
        y=employee_monthly_pivot.loc[emp],
        label=f'Employee {emp}',
        marker='o'
    )
plt.title('Monthly Sentiment Score Trends (Sample Employees)')
plt.xlabel('Month')
plt.ylabel('Sentiment Score')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(employee_monthly_pivot.columns[::3].astype(str), rotation=45)  # Reduce tick density
plt.grid(True)
plt.tight_layout()
plt.show()

Save results 

In [ ]:
employee_monthly_score.to_csv(f'{path}reports/employee_sentiments/employee_monthly_sentiment_score.csv', index=False)
employee_monthly_pivot.to_csv(f'{path}reports/employee_sentiments/employee_monthly_sentiment_score_pivot.csv')

TASK 4. Employee Ranking
For each month, rank employees by their sentiment scores.

In [ ]:
# Convert 'month' back to period for proper sorting
employee_monthly_score['month']=pd.PeriodIndex(employee_monthly_score['month'], freq='M')

Function to get top 3 positive and negative employees per month

In [ ]:
def get_top_employees(df, month):
    month_data = df[df['month'] == month]
    
    # Sort for positive
    top_positive = month_data.sort_values(by=['score', 'from'], ascending=[False, True]).head(3)
    
    # Sort for negative
    top_negative = month_data.sort_values(by=['score', 'from'], ascending=[True, True]).head(3)
    
    return top_positive, top_negative

# Process each month 
ranking_results = []

months=sorted(employee_monthly_score['month'].unique())
for m in months:
    top_pos, top_neg = get_top_employees(employee_monthly_score, m)

    # Top Positive
    df_pos = top_pos.assign(
        month=str(m),
        ranking='Top Positive'
    )[['month', 'from', 'score', 'ranking']]
    
    # Top Negative
    df_neg = top_neg.assign(
        month=str(m),
        ranking='Top Negative'
    )[['month', 'from', 'score', 'ranking']]
    
    # Append both to list
    ranking_results.extend([df_pos, df_neg])

# Combine all into one DataFrame
final_ranking_df = pd.concat(ranking_results, ignore_index=True)

# Preview 
final_ranking_df.head()


# Save rankings to file

In [ ]:
final_ranking_df.to_csv(f'{path}reports/rankings//monthly_employee_rankings.csv', index=False)

Visualize top positive and negative employees for the last month

In [ ]:
last_month = max(months)

last_month_rankings = final_ranking_df[final_ranking_df['month'] == str(last_month)]

last_month_rankings['short_name'] = last_month_rankings['from'].str.split('@').str[0]

plt.figure(figsize=(10,5))
sns.barplot(data=last_month_rankings, x='short_name', y='score', hue='ranking', dodge=False,
            palette={'Top Positive':'green', 'Top Negative':'red'})

plt.title(f'Employee Rankings for {last_month}')
plt.xlabel('Employee')
plt.ylabel('Sentiment Score')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()


Task 5: Flight Risk Identification
 An employee is flagged as **flight risk** if they send **4 or more negative messages in any rolling 30-day window**.

In [ ]:
#Filter only negative message 
negative_msgs=df[df['sentiment']=='Negative'].copy()

# Sort by employee and date for rolling count
negative_msgs = negative_msgs.sort_values(by=['from', 'date'])

 Create rolling 30-day negative message counts for each employee

In [ ]:
flight_risk_flags=[]

for emp_id, group in negative_msgs.groupby('from'):
    group = group.copy()
    group['neg_count_30d'] = group.set_index('date')['sentiment'].rolling('30D').count().values
    
    # Flag if 4 or more negative messages in 30 days
    group['flight_risk_flag'] = group['neg_count_30d'] >= 4
    
    flight_risk_flags.append(group)

# Combine results
flight_risk_df = pd.concat(flight_risk_flags)

# %%
# Get unique employees flagged as flight risk
flight_risk_employees = flight_risk_df[flight_risk_df['flight_risk_flag']]['from'].unique()

print(f"Number of employees flagged as flight risk: {len(flight_risk_employees)}")
print("Flight Risk Employees List:")
print(sorted(flight_risk_employees))

Save Flight risk to csv 

In [ ]:
pd.DataFrame({'from':flight_risk_employees}).to_csv(f"{path}reports/flight_risk/flight_risk_employees.csv", index=False)

Visualize negative message patterns for sample flight risk employees

In [ ]:
sample_risk_employees = flight_risk_employees[:5]


plt.figure(figsize=(12,6))
for emp in sample_risk_employees:
    emp_data = negative_msgs[negative_msgs['from'] == emp]
    plt.plot(emp_data['date'], emp_data.groupby('date').size().cumsum(), marker='o', label=f'Employee {emp}')

plt.title('Cumulative Negative Messages - Sample Flight Risk Employees')
plt.xlabel('Date')
plt.ylabel('Cumulative Negative Messages')
plt.legend()
plt.grid(True)
plt.show()


Task 6: Predictive Modeling -Linear Regression 
Develop a linear regression model to predict the monthly sentiment score of an employee based on:
 - Number of messages in the month
 - Average message length
 - Total word count in the month

In [ ]:
# Import necessary libraries 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Load labeled dataset 

df.head()

In [ ]:
#Drop unnamed: 0 if present 
df=df.drop(columns=['Unnamed: 0'], errors='ignore')

 Feature Engineering
 Aggregate features per employee per month

In [ ]:

# Calculate 'message_length'
df['message_length'] = df['body'].apply(len)

# Calculate 'word_count'
df['word_count'] = df['body'].apply(lambda x: len(str(x).split()))
features = df.groupby(['from', 'month']).agg(
    num_messages=('body', 'count'),
    avg_msg_length=('message_length', 'mean'),
    total_word_count=('word_count', 'sum'),
    sentiment_score=('score', 'sum')  # Target variable
).reset_index()
# Preview the dataset
print(type(features))
features.head()

Encode sentiment labels (Positive=1, Neutral=0, Negative=-1)

In [ ]:


# Assuming 'score' is the desired numerical target.

X = features[['num_messages', 'avg_msg_length', 'total_word_count']]
y = features['sentiment_score']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now, retrain your linear regression and random forest models with X_train, y_train
# and evaluate them on X_test, y_test. You will see more realistic (non-zero) error metrics.

 Baseline: Linear Regression

In [180]:
# Linear Regression Model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predictions
y_pred_lr = lr.predict(X_test)

# Evaluation
mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

print(f"Linear Regression - MAE: {mae_lr:.2f}, RMSE: {rmse_lr:.2f}, R²: {r2_lr:.2f}")


Linear Regression - MAE: 1.54, RMSE: 1.97, R²: 0.67


 Advanced Model: Random Forest Regressor

In [ ]:
# Random Forest Model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_pred_rf = rf.predict(X_test)

# Evaluation
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest - MAE: {mae_rf:.2f}, RMSE: {rmse_rf:.2f}, R²: {r2_rf:.2f}")


 Visualization and Error Analysis

In [ ]:
plt.figure(figsize=(10, 8)) # Increased figure size for better readability with added text

# Scatter plot for Random Forest predictions
plt.scatter(y_test, y_pred_rf, color='blue', alpha=0.6, label='Random Forest')

# Scatter plot for Linear Regression predictions
plt.scatter(y_test, y_pred_lr, color='green', alpha=0.5, label='Linear Regression')

# Ideal Fit line (Actual = Predicted)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Ideal Fit')

# Add Labels and Title
plt.xlabel("Actual Sentiment Score")
plt.ylabel("Predicted Sentiment Score")
plt.title("Actual vs Predicted Sentiment Scores with Model Metrics")

# Add the calculated metrics to the plot
# Adjust coordinates (x, y) as needed to prevent overlap with data points or legend
# Top-left corner is usually good for metrics if the data doesn't clutter it
plt.text(0.5, plt.ylim()[1] * 0.95,  # Top part of the y-axis
         f'Linear Regression:\n MAE: {mae_lr:.2f}\n RMSE: {rmse_lr:.2f}\n R²: {r2_lr:.2f}',
         color='green', fontsize=10, bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

plt.text(plt.xlim()[1] * 0.7, plt.ylim()[1] * 0.1, # Bottom-right corner
         f'Random Forest:\n MAE: {mae_rf:.2f}\n RMSE: {rmse_rf:.2f}\n R²: {r2_rf:.2f}',
         color='blue', fontsize=10, bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))


# Add Legend and Grid
plt.legend()
plt.grid(True)
plt.show()


Linear Regression Actual and Predicted Scores 

In [ ]:
# --- 8. Plotting the Linear Regression Results ---
plt.figure(figsize=(9, 7))

# Scatter plot for Linear Regression predictions
plt.scatter(y_test, y_pred_lr, color='green', alpha=0.6, label='Linear Regression')

# Ideal Fit line (Actual = Predicted)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Ideal Fit')

# Add Labels and Title
plt.xlabel("Actual Sentiment Score")
plt.ylabel("Predicted Sentiment Score")
plt.title("Linear Regression: Actual vs Predicted Sentiment Scores")

# Add the calculated metrics to the plot for Linear Regression
plt.text(0.5, plt.ylim()[1] * 0.95, # Top-left corner
         f'Linear Regression Metrics:\n MAE: {mae_lr:.2f}\n RMSE: {rmse_lr:.2f}\n R²: {r2_lr:.2f}',
         color='green', fontsize=10, bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
